<a href="https://colab.research.google.com/github/vaishuae1999/Train_Machine_Learning_Project/blob/main/train_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Step 1: Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Step 2: Create project folder structure ---
import os

base_dir = "/content/drive/MyDrive/train_project"
raw_video_dir = os.path.join(base_dir, "Raw_video", "DHN-wagon")
processed_video_dir = os.path.join(base_dir, "Processed_Video")

# Create folders if they don’t exist
os.makedirs(raw_video_dir, exist_ok=True)
os.makedirs(processed_video_dir, exist_ok=True)

print("✅ Folders created:")
print("Raw video path:", raw_video_dir)
print("Processed video path:", processed_video_dir)

# --- Step 3: Install dependencies ---
!pip install opencv-python moviepy numpy tqdm ultralytics reportlab

# --- Step 4: Verify installation ---
import cv2, moviepy, numpy, tqdm, ultralytics, reportlab
print("✅ All libraries imported successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Folders created:
Raw video path: /content/drive/MyDrive/train_project/Raw_video/DHN-wagon
Processed video path: /content/drive/MyDrive/train_project/Processed_Video
✅ All libraries imported successfully!


In [ ]:
# ==========================
# INSTALL DEPENDENCIES
# ==========================
!pip install ultralytics reportlab opencv-python-headless

import cv2, os, glob, shutil, numpy as np
from ultralytics import YOLO
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from matplotlib import pyplot as plt

# ==========================
# PATHS
# ==========================
input_folder  = "/content/drive/MyDrive/train_project/Raw_video/DHN-wagon"   # raw videos
work_root     = "/content/drive/MyDrive/train_project/Processed"             # all outputs
frames_root   = os.path.join(work_root, "Frames")
coach_video_root = os.path.join(work_root, "Coach_Videos")
coach_frames_root = os.path.join(work_root, "Coach_Frames")
report_frames_root = os.path.join(work_root, "Report_Frames")
report_root   = os.path.join(work_root, "Reports")

for p in [frames_root, coach_video_root, coach_frames_root, report_frames_root, report_root]:
    os.makedirs(p, exist_ok=True)

# ==========================
# LOAD YOLO MODEL
# ==========================
# Replace 'yolov8n.pt' with your custom-trained model for doors
model = YOLO('yolov8n.pt')

# ==========================
# HELPER: DETECT COACH BOUNDARIES
# ==========================
def detect_coach_boundaries(frame, expected_width=None):
    """Return list of vertical boundary positions for coaches"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # strengthen vertical edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,25))
    edges = cv2.dilate(edges, kernel, iterations=1)

    # vertical projection
    vertical_projection = np.sum(edges, axis=0)
    vp_sm = np.convolve(vertical_projection, np.ones(31)/31, mode='same')

    threshold = vp_sm.max() * 0.25
    boundaries = [i for i,v in enumerate(vp_sm) if v > threshold]

    # group close lines
    clean = []
    for b in boundaries:
        if not clean or b - clean[-1] > 50:
            clean.append(b)

    if expected_width and clean[-1] != expected_width:
        clean.append(expected_width)
    if clean[0] != 0:
        clean = [0] + clean
    return clean

# ==========================
# STEP 1+2: READ VIDEOS & SAVE FRAMES
# ==========================
video_files = glob.glob(os.path.join(input_folder, "*.mp4"))
print(f"Found {len(video_files)} videos.")

for video_path in video_files:
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frame_dir = os.path.join(frames_root, video_name)
    os.makedirs(video_frame_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_no, saved = 0, 0
    sample_frame = None
    while True:
        ret, frame = cap.read()
        if not ret: break
        if frame_no % 10 == 0:
            out_path = os.path.join(video_frame_dir, f"{video_name}_frame_{frame_no}.jpg")
            cv2.imwrite(out_path, frame)
            saved += 1
            if sample_frame is None: sample_frame = frame
        frame_no += 1
    cap.release()
    print(f"Extracted {saved} frames from {video_name}")

    # ==========================
    # STEP 3: DETECT COACHES
    # ==========================
    boundaries = detect_coach_boundaries(sample_frame, expected_width=width)
    coach_count = len(boundaries)-1
    print(f"Detected {coach_count} coaches in {video_name}")

    # ==========================
    # STEP 4: SPLIT INTO COACH VIDEOS
    # ==========================
    cap = cv2.VideoCapture(video_path)
    writers = []
    coach_dir = os.path.join(coach_video_root, video_name)
    os.makedirs(coach_dir, exist_ok=True)

    for i in range(len(boundaries)-1):
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out_file = os.path.join(coach_dir, f"{video_name}_{i+1}.mp4")
        writer = cv2.VideoWriter(out_file, fourcc, fps, (boundaries[i+1]-boundaries[i], height))
        writers.append(writer)

    while True:
        ret, frame = cap.read()
        if not ret: break
        for i in range(len(boundaries)-1):
            crop = frame[:, boundaries[i]:boundaries[i+1]]
            writers[i].write(crop)
    cap.release()
    for w in writers: w.release()
    print(f"Saved {coach_count} coach videos for {video_name}")

    # ==========================
    # STEP 5: EXTRACT FRAMES PER COACH
    # ==========================
    for i in range(len(boundaries)-1):
        coach_video = os.path.join(coach_dir, f"{video_name}_{i+1}.mp4")
        coach_frame_dir = os.path.join(coach_frames_root, f"{video_name}_{i+1}")
        os.makedirs(coach_frame_dir, exist_ok=True)

        cap = cv2.VideoCapture(coach_video)
        frame_no, saved = 0, 0
        while True:
            ret, frame = cap.read()
            if not ret: break
            if frame_no % 10 == 0:
                out_path = os.path.join(coach_frame_dir, f"{video_name}_{i+1}_frame_{frame_no}.jpg")
                cv2.imwrite(out_path, frame)
                saved += 1
            frame_no += 1
        cap.release()
        print(f"Extracted {saved} frames for coach {i+1}")

        # ==========================
        # STEP 6: DOOR DETECTION
        # ==========================
        frames = sorted(glob.glob(os.path.join(coach_frame_dir, "*.jpg")))
        for f in frames:
            results = model(f, conf=0.3)
            results[0].save(filename=f.replace(".jpg","_det.jpg"))

        # ==========================
        # STEP 7: MINIMAL IMAGES (FRONT/MID/REAR)
        # ==========================
        if frames:
            sel = [frames[0], frames[len(frames)//2], frames[-1]]
            report_dir = os.path.join(report_frames_root, f"{video_name}_{i+1}")
            os.makedirs(report_dir, exist_ok=True)
            for sf in sel:
                shutil.copy(sf, os.path.join(report_dir, os.path.basename(sf)))

# ==========================
# STEP 8: GENERATE REPORTS
# ==========================
styles = getSampleStyleSheet()

train_ids = {}
coach_dirs = sorted(glob.glob(os.path.join(report_frames_root, "*")))
for cd in coach_dirs:
    cname = os.path.basename(cd)        # train1_1
    tid = cname.split("_")[0]           # train1
    train_ids.setdefault(tid, []).append(cd)

for tid, coach_dirs in train_ids.items():
    report_path = os.path.join(report_root, f"{tid}_report.pdf")
    doc = SimpleDocTemplate(report_path, pagesize=A4)
    story = []

    story.append(Paragraph(f"Train Report: {tid}", styles['Title']))
    story.append(Spacer(1,20))
    story.append(Paragraph(f"Total Coaches: {len(coach_dirs)}", styles['Heading2']))
    story.append(PageBreak())

    for cd in sorted(coach_dirs):
        cname = os.path.basename(cd)
        story.append(Paragraph(f"Coach {cname}", styles['Heading2']))
        story.append(Spacer(1,10))

        imgs = sorted(glob.glob(os.path.join(cd,"*.jpg")))
        for im in imgs:
            try:
                story.append(Image(im, width=200, height=120))
                story.append(Spacer(1,10))
            except: pass
        story.append(PageBreak())

    doc.build(story)
    print(f"✅ Report saved: {report_path}")


Streaming output truncated to the last 5000 lines.

image 1/1 /content/drive/MyDrive/train_project/Processed/Coach_Frames/DHN-upper-side-view-2025-08-31-11-28-15-377_25/DHN-upper-side-view-2025-08-31-11-28-15-377_25_frame_1930.jpg: 640x64 (no detections), 92.6ms
Speed: 0.8ms preprocess, 92.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 64)

image 1/1 /content/drive/MyDrive/train_project/Processed/Coach_Frames/DHN-upper-side-view-2025-08-31-11-28-15-377_25/DHN-upper-side-view-2025-08-31-11-28-15-377_25_frame_1940.jpg: 640x64 (no detections), 67.9ms
Speed: 0.8ms preprocess, 67.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 64)

image 1/1 /content/drive/MyDrive/train_project/Processed/Coach_Frames/DHN-upper-side-view-2025-08-31-11-28-15-377_25/DHN-upper-side-view-2025-08-31-11-28-15-377_25_frame_1950.jpg: 640x64 (no detections), 105.0ms
Speed: 0.8ms preprocess, 105.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 64)

image 1/1 /content/dri